In [38]:
def initiateGenerator(path, batchSize, imageSize):
    base_path = path
    print("\nTotal : ", end=" ")
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(batch_size=batchSize, 
                                                                        directory=base_path+"/"+"train")
    train_datagen = ImageDataGenerator(rescale=1./255)

    print("\nFor Training : ", end=" ")
    train_generator = train_datagen.flow_from_directory(
        base_path+"/"+"train",
        target_size=(imageSize, imageSize),
        batch_size=batchSize,
        class_mode='categorical', subset='training')

    print("\nFor Val : ", end=" ")
    valid_datagen = ImageDataGenerator(rescale=1./255)
    validation_generator = valid_datagen.flow_from_directory(
        base_path+"/"+"valid",
#                 base_path+"/"+"train",

#         base_path + "/" + "Training",
        target_size=(imageSize, imageSize),
        batch_size=batchSize,
        class_mode='categorical',shuffle=False)
    
    print("\nFor Test : ", end=" ")
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
#         base_path+"/"+"Testing",
        base_path + "/" + "test",
        target_size=(imageSize, imageSize),
        batch_size=batchSize,
        class_mode='categorical', shuffle=False)
    class_names = train_dataset.class_names
    noOfClasses = len(class_names)
    print("\nNo of Classes : ", noOfClasses)
    print("Classes : ", class_names)

        
    return noOfClasses,class_names, train_generator, validation_generator, test_generator

In [39]:
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import layers, models
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Rescaling, Conv2D, MaxPool2D, Dropout, Dense, Flatten, BatchNormalization
import numpy as np
from sklearn.utils import class_weight
import pandas as pd

imageSize = 64
batchSize = 32
mpath = r"E:\Downloads\bird-species"

noOfClasses, class_names, train_generator, validation_generator, test_generator = initiateGenerator(mpath, batchSize=batchSize, imageSize=imageSize)
INPUT_SHAPE = (imageSize, imageSize, 3)
KERNEL_SIZE = (3, 3)
class_weight = class_weight.compute_class_weight(
                class_weight='balanced',
                classes=np.unique(train_generator.classes), 
                y=train_generator.classes)
class_weight = {x : class_weight[x] for x in range(len(class_weight))}



Total :  Found 3208 files belonging to 20 classes.

For Training :  Found 3208 images belonging to 20 classes.

For Val :  Found 100 images belonging to 20 classes.

For Test :  Found 100 images belonging to 20 classes.

No of Classes :  20
Classes :  ['ABBOTTS BABBLER', 'ABBOTTS BOOBY', 'ABYSSINIAN GROUND HORNBILL', 'AFRICAN CROWNED CRANE', 'AFRICAN EMERALD CUCKOO', 'AFRICAN FIREFINCH', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN PYGMY GOOSE', 'ALBATROSS', 'ALBERTS TOWHEE', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'ALTAMIRA YELLOWTHROAT', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN FLAMINGO', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL']


In [40]:
def hyperTune(**kwargs):

    annealer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, verbose=0, min_lr=1e-5, mode="max")
    early = EarlyStopping(monitor="val_accuracy", patience=3, mode="max", verbose=0)
    train_generator.batch_size = kwargs["batch_size"]
    validation_generator.batch_size = kwargs["batch_size"]
    test_generator.batch_size = kwargs["batch_size"]

    model = Sequential()
    model.add(Conv2D(filters=kwargs["Layer1_filter"], kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation=kwargs["Layer1_act"], padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=kwargs["Layer2_filter"], kernel_size=KERNEL_SIZE, activation=kwargs["Layer2_act"], padding='same'))
    model.add(BatchNormalization())
    # Pooling layer
    model.add(MaxPool2D(pool_size=(2, 2)))
    # Dropout layers
    model.add(Dropout(kwargs["Drop1"]))

    model.add(Conv2D(filters=kwargs["Layer3_filter"], kernel_size=KERNEL_SIZE, activation=kwargs["Layer3_act"], padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=kwargs["Layer4_filter"], kernel_size=KERNEL_SIZE, activation=kwargs["Layer4_act"], padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(kwargs["Drop2"]))

    model.add(Conv2D(filters=kwargs["Layer5_filter"], kernel_size=KERNEL_SIZE, activation=kwargs["Layer5_act"], padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=kwargs["Layer6_filter"], kernel_size=KERNEL_SIZE, activation=kwargs["Layer6_act"], padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(kwargs["Drop3"]))

    model.add(Flatten())
    model.add(Dense(kwargs["Layer7_units"], activation=kwargs["Layer7_act"]))
    model.add(Dropout(kwargs["Drop4"]))
    model.add(Dense(noOfClasses, activation='softmax'))
    if kwargs["optimizer"] == "sgd":
        optimizer = tf.keras.optimizers.SGD(kwargs["learning_rate"])
    elif kwargs["optimizer"] == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(kwargs["learning_rate"])
    elif kwargs["optimizer"] == "adam":
        optimizer = tf.keras.optimizers.Adam(kwargs["learning_rate"])
    elif kwargs["optimizer"] == "nadam":
        optimizer = tf.keras.optimizers.Nadam(kwargs["learning_rate"])
    elif kwargs["optimizer"] == "adadelta":
        optimizer = tf.keras.optimizers.Adadelta(kwargs["learning_rate"])
    elif kwargs["optimizer"] == "adagrad":
        optimizer = tf.keras.optimizers.Adagrad(kwargs["learning_rate"])
    # Compile the model
    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    # Train the model
    model.fit(train_generator, epochs=kwargs["epochs"], 
            class_weight=class_weight, verbose=0,
            validation_data=validation_generator,
            callbacks=[annealer, early])

    test_loss, test_acc = model.evaluate(test_generator, verbose=0)
    del model
    return test_acc

In [41]:
hp = pd.read_excel(r"E:\New folder\best_hyperparameter.xls")
display(hp)

,Hyperparameters,BO + DE,BO + PSO,BO + HS,BO + ADE,BO + APSO,BO + AHS,DE,PSO,HS,ADE,APSO,AHS,Base
0,Layer1_filter,64,32,32,32,32,64,64,32,64,64,32,32,32
1,Layer1_act,gelu,gelu,relu,relu,selu,leaky_relu,leaky_relu,relu,selu,relu,selu,selu,relu
2,Layer2_filter,64,64,32,32,32,32,32,64,64,32,64,64,32
3,Layer2_act,elu,selu,elu,gelu,elu,gelu,elu,leaky_relu,selu,gelu,elu,selu,relu
4,Drop1,0.25,0.35,0.45,0.5,0.35,0.45,0.5,0.45,0.25,0.25,0.4,0.25,0.25
5,Layer3_filter,32,32,64,128,32,64,128,128,128,128,32,32,64
6,Layer3_act,selu,selu,leaky_relu,relu,leaky_relu,gelu,gelu,elu,leaky_relu,selu,gelu,relu,relu
7,Layer4_filter,32,128,32,64,128,32,128,128,64,32,128,32,64
8,Layer4_act,relu6,gelu,leaky_relu,leaky_relu,relu6,selu,leaky_relu,gelu,relu6,leaky_relu,elu,relu6,relu
9,Drop2,0.25,0.4,0.3,0.35,0.25,0.35,0.25,0.35,0.3,0.25,0.45,0.25,0.25


In [42]:
opt_algo = hp.columns[1:-1].to_list()
hp_list = hp["Hyperparameters"].to_list()[:-2]

params = {}

for algo in opt_algo:
    hp_dict = {hp_list[idx]:idx for idx in range(len(hp_list))}
    t = hp[algo][:-2].to_dict()
    for idx, key in enumerate(hp_dict.keys()):
        hp_dict[key] = t[idx]
    params[algo] = hp_dict

In [43]:
import tqdm
import numpy as np
result = {}

for algo, v in zip(opt_algo, params.values()):
    print("Current Algorithm : ", algo)
    print("Hyperparameters : ", v)
    acc = []
    iteration = tqdm.tqdm(range(100), desc=algo)
    best_acc = -np.inf
    for _ in iteration:
        iteration.set_postfix_str(f"best_acc : {round(best_acc, 5)}")

        acc_ = hyperTune(**v)
        acc.append(acc_)
        print(acc_)
        if acc_ > best_acc:
            best_acc = acc_

    result[algo] = acc
    pd.DataFrame({algo:acc}).to_csv(f"{algo}.csv", index=False)
pd.DataFrame(result).to_csv("result.csv", index=False)


Current Algorithm :  ADE
Hyperparameters :  {'Layer1_filter': 64, 'Layer1_act': 'relu', 'Layer2_filter': 32, 'Layer2_act': 'gelu', 'Drop1': 0.25, 'Layer3_filter': 128, 'Layer3_act': 'selu', 'Layer4_filter': 32, 'Layer4_act': 'leaky_relu', 'Drop2': 0.25, 'Layer5_filter': 64, 'Layer5_act': 'relu6', 'Layer6_filter': 64, 'Layer6_act': 'relu6', 'Drop3': 0.25, 'Layer7_units': 1024, 'Layer7_act': 'relu', 'Drop4': 0.5, 'optimizer': 'adam', 'epochs': 50, 'batch_size': 64, 'learning_rate': 0.001, 'Accuracy': 0.939999997615814, 'Mean Accuracy across 100 runs': nan, 'Max Accuracy across 100 runs': nan}


ADE:   1%|          | 1/100 [00:16<26:40, 16.16s/it, best_acc : 0.1] 

0.10000000149011612


ADE:   2%|▏         | 2/100 [00:32<26:10, 16.03s/it, best_acc : 0.1]

0.07999999821186066


...

0.9100000262260437


ADE:   5%|▌         | 5/100 [03:15<1:19:09, 50.00s/it, best_acc : 0.91]

0.8700000047683716


ADE:   5%|▌         | 5/100 [03:22<1:04:16, 40.59s/it, best_acc : 0.91]


KeyboardInterrupt: 